# 数据预处理

1. 读取附件1中数据
2. 去除无用属性
3. 使用箱线图检测异常值并去除
4. 去除实际金额为0的数据
5. 将附件2中的商品信息添加到附件1中
6. 按地点分割数据

## 0. 导入模块

In [24]:
import pandas as pd
import numpy as np

## 1. 读取数据

In [25]:
f=open('../数据/附件1.csv')
all_sales_data=pd.read_csv(f)
all_sales_data.head()

,订单号,设备ID,应付金额,实际金额,商品,支付时间,地点,状态,提现
0,DD201708167493663618499909784,E43A6E078A07631,4.5,4.5,68g好丽友巧克力派2枚,2017/1/1 00:53,D,已出货未退款,已提现
1,DD201708167493663555814061164,E43A6E078A04172,3.0,3.0,40g双汇玉米热狗肠,2017/1/1 01:33,A,已出货未退款,已提现
2,DD201708167493578526890939886,E43A6E078A06874,5.5,5.5,430g泰奇八宝粥,2017/1/1 08:45,E,已出货未退款,已提现
3,DD201708167493683507186615837,E43A6E078A04228,5.0,5.0,48g好丽友薯愿香烤原味,2017/1/1 09:05,C,已出货未退款,已提现
4,DD201708167493759548618252006,E43A6E078A04134,3.0,3.0,600ml可口可乐,2017/1/1 09:41,B,已出货未退款,已提现


## 2. 去除无用属性

In [26]:
#去除属性包括订单号、设备ID、提现、状态、应付金额
#将属性名“实际金额”修改为“金额”
all_sales_data.drop(columns=["订单号","设备ID","提现","状态","应付金额"],inplace=True)
all_sales_data.rename(columns={"实际金额":"金额"},inplace=True)
all_sales_data.head()

,金额,商品,支付时间,地点
0,4.5,68g好丽友巧克力派2枚,2017/1/1 00:53,D
1,3.0,40g双汇玉米热狗肠,2017/1/1 01:33,A
2,5.5,430g泰奇八宝粥,2017/1/1 08:45,E
3,5.0,48g好丽友薯愿香烤原味,2017/1/1 09:05,C
4,3.0,600ml可口可乐,2017/1/1 09:41,B


## 3.使用箱线图检测异常值并去除

In [27]:
Group=all_sales_data.groupby('商品')
#使用箱线图法检查并找出异常值，并提取异常值数据的索引
def Outliers(a_list):
    outlier_indices = list()
    Q1 = np.percentile(a_list, 25)
    Q3 = np.percentile(a_list,75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR
    outlier_list_col = a_list[(a_list < Q1 - outlier_step) | (a_list > Q3 + outlier_step )]
    return outlier_list_col
Out_Index=list()
for i in Group.groups:
    grp = Group.get_group(i)
    outlier = Outliers(grp['金额'])
    Out_Index.append(outlier.index)

#删除异常数据
for i in Out_Index:
    all_sales_data.drop(labels = i, axis = 0, inplace = True)

## 3. 去除金额为0的数据

In [28]:
data_0=all_sales_data[all_sales_data['金额']==0]  #金额为0的商品数据
data_0.head()

,金额,商品,支付时间,地点
408,0.0,85g新麦潮北海道吐司,2017/1/9 08:00,D
1037,0.0,40g双汇玉米热狗肠,2017/1/10 18:24,C
1144,0.0,85g新麦潮北海道吐司,2017/1/10 23:36,C
1495,0.0,85g新麦潮北海道吐司,2017/1/30 15:39,D
6604,0.0,85g新麦潮北海道吐司,2017/4/28 02:35,A


In [29]:
#去除金额为0的商品数据
data_index=data_0.index
all_sales_data.drop(index=data_index,inplace=True)

## 5. 添加商品信息

In [30]:
f=open('../数据/附件2.csv')
info_data=pd.read_csv(f)
info_data.head()

,商品,大类,二级类
0,100g*5瓶益力多,饮料,乳制品
1,100g越南LIPO奶味面包干,非饮料,饼干糕点
2,10g卫龙亲嘴烧香辣味,非饮料,肉干/豆制品/蛋
3,10g越南LIPO奶味面包干,非饮料,饼干糕点
4,110g顺宝九制话梅,非饮料,蜜饯/果干


In [31]:
first_type=dict(map(lambda x,y:[x,y],list(info_data['商品']),list(info_data['大类'])))
second_type=dict(map(lambda x,y:[x,y],list(info_data['商品']),list(info_data['二级类'])))
all_sales_data['商品大类']=all_sales_data['商品'].map(first_type)
all_sales_data['商品二级类']=all_sales_data['商品'].map(second_type)
all_sales_data.head()

,金额,商品,支付时间,地点,商品大类,商品二级类
0,4.5,68g好丽友巧克力派2枚,2017/1/1 00:53,D,非饮料,饼干糕点
1,3.0,40g双汇玉米热狗肠,2017/1/1 01:33,A,非饮料,肉干/豆制品/蛋
3,5.0,48g好丽友薯愿香烤原味,2017/1/1 09:05,C,非饮料,膨化食品
4,3.0,600ml可口可乐,2017/1/1 09:41,B,饮料,碳酸饮料
5,4.5,营养快线,2017/1/1 09:41,B,饮料,乳制品


In [32]:
#初步处理的数据保存为csv文件，以供后续分析使用
all_sales_data.to_csv('../数据/初步处理数据.csv', encoding='gbk',index=None)

## 6. 按地点提取数据（任务1.1）

In [33]:
place=['A','B','C','D','E']
for pla in place:
    place_data=all_sales_data[all_sales_data['地点']==pla]
    place_data.to_csv('../数据/分割数据/task1-1'+pla+'.csv', encoding='gbk',index=None)